<a href="https://colab.research.google.com/github/jihun0423/Dacon-Hansol-img-classification/blob/main/WeightedRandomSampler_%26_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import torchvision.transforms as transforms
from torchvision import datasets

In [ ]:
base_dir = '/content/gdrive/MyDrive/open (2)/'

In [ ]:
train_folder = glob(base_dir + 'train/*')

train_path = []
for folder in train_folder:
    tmp = glob(folder + '/*')
    train_path += tmp

In [ ]:
train_df = pd.DataFrame(train_path, columns=['path'])
train_df['label'] = train_df['path'].apply(lambda x: x.split('/')[-2])
train_df


,path,label
0,/content/gdrive/MyDrive/open (2)/train/가구수ᄌ...,가구수정
1,/content/gdrive/MyDrive/open (2)/train/가구수ᄌ...,가구수정
2,/content/gdrive/MyDrive/open (2)/train/가구수ᄌ...,가구수정
3,/content/gdrive/MyDrive/open (2)/train/가구수ᄌ...,가구수정
4,/content/gdrive/MyDrive/open (2)/train/가구수ᄌ...,가구수정
...,...,...
3452,/content/gdrive/MyDrive/open (2)/train/틈새과...,틈새과다
3453,/content/gdrive/MyDrive/open (2)/train/틈새과...,틈새과다
3454,/content/gdrive/MyDrive/open (2)/train/틈새과...,틈새과다
3455,/content/gdrive/MyDrive/open (2)/train/틈새과...,틈새과다


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train_df['label'] = le.fit_transform(train_df['label']) # 라벨 인코딩

In [ ]:
train_df['label'].value_counts()

18    1405
10     595
1      307
3      210
15     162
2      145
11     142
7      130
6       99
9       57
5       54
17      51
14      27
12      22
13      17
4       14
0       12
16       5
8        3
Name: label, dtype: int64

# WeightedRandomSampler

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

train_labels = pd.DataFrame(df['label'])
class_counts = train_labels.value_counts(sort=False).to_list() 
num_samples = sum(class_counts) # 전체 데이터 갯수
labels = [train_labels]
#클래스별 가중치 부여
class_weights = [num_samples / class_counts[i] for i in range(len(class_counts))] #해당 레이블마다의 가중치 비율
weights = [class_weights[labels[i]] for i in range(int(num_samples))] # 해당 데이터의 label에 해당되는 가중치
weights = []
for i in range(int(num_samples)):
  weights.append(class_weights[int(train_labels.iloc[i,:])])
sampler = WeightedRandomSampler(torch.DoubleTensor(weights), int(num_samples))



In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image)
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [ ]:
class_weights

[288.0833333333333,
 11.260586319218241,
 23.841379310344827,
 16.461904761904762,
 246.92857142857142,
 64.01851851851852,
 34.91919191919192,
 26.592307692307692,
 1152.3333333333333,
 60.64912280701754,
 5.810084033613445,
 24.345070422535212,
 157.13636363636363,
 203.35294117647058,
 128.03703703703704,
 21.339506172839506,
 691.4,
 67.7843137254902,
 2.4604982206405692]

In [ ]:
pd.DataFrame(weights).value_counts()                          # 각 클래스 별 가중치. 데이터 개수가 많은 라벨일수록 작게, 데이터 개수가 적은 라벨일수록 가중치를 크게 부여해 데이터가 골고루 추출될 수 있도록 함

2.460498       1405
5.810084        595
11.260586       307
16.461905       210
21.339506       162
23.841379       145
24.345070       142
26.592308       130
34.919192        99
60.649123        57
64.018519        54
67.784314        51
128.037037       27
157.136364       22
203.352941       17
246.928571       14
288.083333       12
691.400000        5
1152.333333       3
dtype: int64

In [ ]:
import torchvision.transforms as transforms
bg_color = (146.9594577448965, 151.0021480420162, 153.14668947889484)
augmentate = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((256, 256)),
        transforms.RandomCrop((240, 240)),
        transforms.RandomHorizontalFlip(p=0.7),
        transforms.Resize((224, 224)),
    ])                                                      # 이미지 변형 방법 설정. 이미지들을 확인한 결과, RandomRotate와 RandomVerticalFlip을 사용할 경우 분류가 불가능한 라벨들이 있어서 배제.

In [ ]:
from torchvision.transforms.functional import to_pil_image                        # 변형한 이미지 텐서를 이미지 파일로 저장하기 위해 임포트

In [ ]:
df = train_df
train_augmentate = CustomDataset(df['img_path'].values, df['label'].values, augmentate)
augmentate_loader = DataLoader(train_augmentate, batch_size = len(df), shuffle=False, num_workers=8, sampler=sampler)                 # WeightedRandomSampler을 이용한 DataLoader

In [ ]:
l = []    # 생성한 이미지의 라벨을 저장하기 위한 리스트
aug_img = []    # 생성한 이미지 리스트
for i in range(0,3):        # 데이터를 3배로 늘리고 싶기에 에폭을 3으로 설정
  for j, (inputs,labels) in enumerate(augmentate_loader):
    for k in tqdm(range(0,len(df))):
      aug_img.append(to_pil_image(inputs[k]))
      l.append(labels[k])

Exception ignored in: <function tqdm.__del__ at 0x7f5a9ef043a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/tqdm/std.py", line 1145, in __del__
    self.close()
  File "/usr/local/lib/python3.9/dist-packages/tqdm/notebook.py", line 283, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm' object has no attribute 'disp'


  0%|          | 0/3457 [00:00<?, ?it/s]

  0%|          | 0/3457 [00:00<?, ?it/s]

  0%|          | 0/3457 [00:00<?, ?it/s]

In [ ]:
l=pd.DataFrame(l).astype(int)  

In [ ]:
pd.DataFrame(l).astype(int).value_counts()              # 생성된 이미지들의 라벨 분류. 골고루 생성된 것을 확인 할 수 있다.

15    569
5     560
7     560
9     559
10    559
2     559
11    558
12    556
1     554
13    550
0     549
17    548
8     543
4     532
14    531
3     524
18    522
6     519
16    519
dtype: int64

In [ ]:
import os
for i in range(0,19):
  os.mkdir("/content/gdrive/MyDrive/open (2)/aug/"+f'{i}')               # 이미지 저장을 위해 폴더 생성

In [ ]:
for i in range(len(l)):
  aug_img[i].save('/content/gdrive/MyDrive/open (2)/aug/'+f'{int(l.iloc[i,:])}/'+f'{i}.jpg')            # 생성한 이미지들을 라벨에 맞는 폴더에 저장